# LabChronicle — QC entry + (dummy) analysis

This notebook creates a **QC session YAML** from the repo's fully documented template and stores it under `qc/sessions/<microscope>/<YYYY>/`.

**Rules enforced here**:
- Humans enter **inputs** and **raw measurements** only.
- Pass/fail is **not** set here (reserved for CI/GitHub Actions in the `evaluation` block).
- Laser measurements support **linearity** (setpoint→power) and **stability** (time→power), with an explicit `measurement_position`.

> Tip: If you are running from the `notebooks/` directory, this notebook automatically adds `../src` to `sys.path`.


In [ ]:
from __future__ import annotations

from pathlib import Path
import sys
import os
from datetime import datetime, timezone
import copy

# --- repo import path ---
repo_root = Path.cwd().resolve()
if not (repo_root / "src").exists():
    repo_root = repo_root.parent
sys.path.insert(0, str(repo_root / "src"))

import yaml
import ipywidgets as W
from IPython.display import display, Markdown, clear_output

from aic_chronicle.yaml_io import load_template, save_yaml
from aic_chronicle.ids import make_qc_id, slugify
from aic_chronicle import github_ops
from aic_chronicle import widgets as AW
from aic_chronicle import qc_dummy_analysis as QA

TEMPLATE_PATH = repo_root / "templates" / "QC_template.yaml"
INSTRUMENTS_DIR = repo_root / "instruments"


## 1) Select microscope
Choose a microscope from `instruments/`. The notebook will display a summary and the extracted light sources.


In [ ]:
mic_dd, get_selection = AW.instrument_selector(INSTRUMENTS_DIR)
mic_out = W.Output()

def _render_instrument(*_):
    with mic_out:
        clear_output()
        sel = get_selection()
        if sel is None:
            print("No instrument selected.")
            return
        print(f"Instrument file: {sel.metadata.filepath.name}")
        print(f"Derived instrument_id (used for ledgers): {sel.metadata.instrument_id}")
        print(f"Display name: {sel.metadata.display_name}")
        print("")
        print("Extracted light sources:")
        if sel.light_sources:
            for s in sel.light_sources:
                w = s.get("wavelength_nm")
                k = s.get("kind") or "source"
                print(f"  - {s.get('label')}  | kind={k} | wavelength_nm={w}")
        else:
            print("  (none found in YAML — you can still add lasers manually below)")

_render_instrument()
mic_dd.observe(_render_instrument, "value")
display(W.VBox([mic_dd, mic_out]))


## 2) Create new QC session
Fill in session metadata, then click **Initialize QC session**. This loads the QC template into memory and generates a `qc_id`.


In [ ]:
suite = AW.text_widget("QC suite", placeholder="e.g., monthly_qc", value="monthly_qc", width="45%")
started_utc = AW.datetime_widget("started_utc")
ended_utc = AW.text_widget("ended_utc (optional)", placeholder="YYYY-MM-DDThh:mm:ssZ", value="", width="55%")
performed_by = AW.text_widget("performed_by", placeholder="name or user id", value="", width="55%")
reason = AW.text_widget("reason", placeholder="monthly_qc / post_service / troubleshooting / ...", value="monthly_qc", width="55%")
summary = AW.notes_widget("summary")
notes = AW.notes_widget("notes")

init_btn = W.Button(description="Initialize QC session", button_style="success")
qc_id_lbl = W.HTML("<b>qc_id:</b> <code>(not initialized)</code>")
init_out = W.Output()

state = {
    "qc": None,
    "qc_id": None,
    "uploaded_files": {},   # filename -> bytes (from editors)
    "image_upload": None,   # (filename, bytes)
}

def _blank_evaluation_block(qc: dict) -> None:
    # Keep keys, but blank values (CI will fill them)
    qc["evaluation"] = {
        "benchmark_ref": "",
        "evaluated_utc": "",
        "evaluated_by": "",
        "overall_status": "",
        "results": [],
    }

def _blank_computed(qc: dict) -> None:
    qc["computed_provenance"] = {
        "pipeline": "",
        "version": "",
        "git_commit": "",
        "run_utc": "",
    }
    qc["metrics_computed"] = []

def _init_session(_):
    with init_out:
        clear_output()
        sel = get_selection()
        if sel is None:
            print("Select a microscope first.")
            return

        qc = load_template(TEMPLATE_PATH)

        # Basic identity fields
        qc["microscope"] = sel.metadata.instrument_id
        qc["started_utc"] = started_utc.value.strip() or AW.utc_now_iso()
        if ended_utc.value.strip():
            qc["ended_utc"] = ended_utc.value.strip()
        else:
            qc["ended_utc"] = ""
        qc["performed_by"] = performed_by.value.strip()
        qc["reason"] = reason.value.strip() or suite.value.strip() or "qc"
        qc["summary"] = summary.value
        qc["notes"] = notes.value

        qc_id = make_qc_id(sel.metadata.instrument_id, qc["started_utc"], suite.value.strip() or "qc")
        qc["qc_id"] = qc_id

        # Clean script-owned sections for a new session
        _blank_computed(qc)
        _blank_evaluation_block(qc)

        # Ensure human-entered structures exist
        qc.setdefault("inputs_human", [])
        qc.setdefault("laser_inputs_human", {})
        qc.setdefault("performed", [])
        qc.setdefault("artifacts", [])

        state["qc"] = qc
        state["qc_id"] = qc_id

        qc_id_lbl.value = f"<b>qc_id:</b> <code>{qc_id}</code>"
        print("QC session initialized. Continue to laser inputs and analysis selection below.")

init_btn.on_click(_init_session)

display(W.VBox([W.HBox([suite, started_utc]), ended_utc, performed_by, reason, summary, notes, init_btn, qc_id_lbl, init_out]))


## 3) Laser manual entry
Enter the measurement position and add **linearity** and/or **stability** measurements.

- `measurement_position` must be one of: `at_objective | pre_objective | fiber_output`.
- For each laser, you can enter a small number of points inline **or** attach a CSV and store it as an artifact.


In [ ]:
laser_out = W.Output()

measurement_position = W.Dropdown(
    options=[("at_objective", "at_objective"), ("pre_objective", "pre_objective"), ("fiber_output", "fiber_output")],
    value="at_objective",
    description="measurement_position:",
    layout=W.Layout(width="55%")
)
measurement_position_details = AW.notes_widget("measurement_position_details")

pm_model = AW.text_widget("power meter model", value="", width="55%")
pm_serial = AW.text_widget("serial (optional)", value="", width="55%")
pm_cal = AW.text_widget("calibration_due", placeholder="YYYY-MM-DD or free text", value="", width="55%")
pm_int = W.FloatText(value=1.0, description="integration_time_s:", layout=W.Layout(width="55%"))
pm_notes = AW.notes_widget("power meter notes")

# Build laser option list from instrument light sources (prefer wavelengths if present)
sel = get_selection()
laser_options = []
if sel and sel.light_sources:
    for s in sel.light_sources:
        w = s.get("wavelength_nm")
        if w is not None:
            laser_options.append(str(int(w)))
# de-duplicate preserving order
seen = set()
laser_options = [x for x in laser_options if not (x in seen or seen.add(x))]
if not laser_options:
    laser_options = ["488", "561"]  # sensible defaults; user can edit series IDs manually

linearity_widget, get_linearity, get_linearity_files = AW.laser_linearity_editor(laser_options)
stability_widget, get_stability, get_stability_files = AW.laser_stability_editor(laser_options)

apply_lasers_btn = W.Button(description="Apply laser entries to QC YAML", button_style="success")

def _apply_lasers(_):
    with laser_out:
        clear_output()
        if state["qc"] is None:
            print("Initialize a QC session first.")
            return
        qc = state["qc"]
        qc["laser_inputs_human"] = qc.get("laser_inputs_human") or {}
        qc["laser_inputs_human"]["measurement_position"] = measurement_position.value
        qc["laser_inputs_human"]["measurement_position_details"] = measurement_position_details.value

        qc["laser_inputs_human"]["power_meter"] = {
            "model": pm_model.value,
            "serial": pm_serial.value,
            "calibration_due": pm_cal.value,
            "integration_time_s": float(pm_int.value),
            "notes": pm_notes.value,
        }

        qc["laser_inputs_human"]["linearity_series"] = get_linearity()
        qc["laser_inputs_human"]["stability_series"] = get_stability()

        # Collect uploaded files for saving later
        files = {}
        files.update(get_linearity_files())
        files.update(get_stability_files())
        state["uploaded_files"] = files

        print("Laser inputs updated in QC YAML (in memory).")

apply_lasers_btn.on_click(_apply_lasers)

display(W.VBox([
    measurement_position,
    measurement_position_details,
    W.HTML("<h4>Power meter</h4>"),
    pm_model, pm_serial, pm_cal, pm_int, pm_notes,
    W.HTML("<hr/>"),
    linearity_widget,
    stability_widget,
    apply_lasers_btn,
    laser_out
]))


## 4) Target an image (optional)
Attach a local image **or** provide an OMERO URI/reference. The notebook stores it as an artifact reference.


In [ ]:
img_out = W.Output()

img_upload = AW.file_upload_widget("Upload image", accept=".tif,.tiff,.png,.jpg,.jpeg", multiple=False)
omero_uri = AW.text_widget("OMERO URI / remote ref", placeholder="omero://.../image/<id>", value="", width="90%")
img_desc = AW.text_widget("description", placeholder="What is this image used for?", value="QC target image", width="90%")

apply_img_btn = W.Button(description="Apply image reference", button_style="success")

def _apply_image(_):
    with img_out:
        clear_output()
        if state["qc"] is None:
            print("Initialize a QC session first.")
            return
        qc = state["qc"]
        qc.setdefault("artifacts", [])

        artifact_id = "art_target_image"
        uri = ""
        if img_upload.value:
            for fn, meta in img_upload.value.items():
                state["image_upload"] = (fn, meta["content"])
                # uri will be set at save time (repo-relative path)
                uri = f"(pending save) {fn}"
                break
        elif omero_uri.value.strip():
            uri = omero_uri.value.strip()
        else:
            print("No image selected (that's OK).")
            return

        # Upsert artifact
        qc["artifacts"] = [a for a in qc["artifacts"] if a.get("artifact_id") != artifact_id]
        qc["artifacts"].append({
            "artifact_id": artifact_id,
            "role": "raw_image",
            "uri": uri,
            "description": img_desc.value.strip() or "QC target image",
        })

        print(f"Stored artifact reference: {artifact_id} -> {uri}")
        if img_upload.value:
            print("Local image will be written into the repo when you click 'Save QC YAML'.")

apply_img_btn.on_click(_apply_image)

display(W.VBox([img_upload, omero_uri, img_desc, apply_img_btn, img_out]))


## 5) Choose analyses to run (dummy for now)
Select which analyses to compute. Clicking **Run analysis** fills `computed_provenance` and `metrics_computed`.

- Laser metrics are computed from your entered series where possible.
- PSF / alignment / stage metrics are deterministic dummy placeholders.


In [ ]:
analysis_out = W.Output()

analysis_sel = W.SelectMultiple(
    options=[("Laser metrics", "laser"), ("PSF / resolution", "psf"), ("Alignment", "alignment"), ("Stage repeatability", "stage")],
    value=("laser",),
    description="Analyses:",
    layout=W.Layout(width="55%", height="120px")
)
run_btn = W.Button(description="Run analysis", button_style="primary")

def _run_analysis(_):
    with analysis_out:
        clear_output()
        if state["qc"] is None:
            print("Initialize a QC session first.")
            return
        qc = state["qc"]
        qc_id = qc.get("qc_id", "qc_unknown")
        metrics = []

        if "laser" in analysis_sel.value:
            metrics.extend(QA.compute_laser_metrics(qc.get("laser_inputs_human", {})))

        if "psf" in analysis_sel.value:
            metrics.extend(QA.analyze_psf(qc_id))

        if "alignment" in analysis_sel.value:
            metrics.extend(QA.analyze_alignment(qc_id))

        if "stage" in analysis_sel.value:
            metrics.extend(QA.analyze_stage_repeatability(qc_id))

        qc["computed_provenance"] = QA.make_computed_provenance(pipeline="qc_notebook_dummy", version="0.1.0", git_commit="")
        qc["metrics_computed"] = metrics

        # Update performed list (narrative only; no pass/fail)
        qc["performed"] = []
        for key in analysis_sel.value:
            qc["performed"].append({"qc_type": key if key != "laser" else "laser_power", "details": "Recorded in this notebook.", "artifacts": []})

        print(f"Computed {len(metrics)} metrics.")
        for m in metrics[:12]:
            print(f"  - {m['metric_id']}: {m['value']} {m.get('unit','')}")
        if len(metrics) > 12:
            print("  ...")

run_btn.on_click(_run_analysis)

display(W.VBox([analysis_sel, run_btn, analysis_out]))


## 6) Save QC YAML (+ optional git commit/push)
This writes:
- QC YAML to: `qc/sessions/<microscope>/<YYYY>/<qc_id>.yaml`
- Any uploaded artifacts to: `qc/sessions/<microscope>/<YYYY>/artifacts/<qc_id>/...`

If this repo is a **git clone** (contains a `.git` directory), you can optionally create a branch, commit, and push.


In [ ]:
save_out = W.Output()

preview = W.Textarea(value="", description="YAML preview:", layout=W.Layout(width="95%", height="280px"))
preview.disabled = True

refresh_preview_btn = W.Button(description="Refresh preview", button_style="info")
save_btn = W.Button(description="Save QC YAML", button_style="success")

def _yaml_preview_text(qc: dict) -> str:
    return yaml.safe_dump(qc, sort_keys=False, allow_unicode=True, default_flow_style=False, width=120)

def _refresh_preview(_=None):
    with save_out:
        clear_output()
        if state["qc"] is None:
            print("Initialize a QC session first.")
            preview.value = ""
            return
        preview.value = _yaml_preview_text(state["qc"])
        print("Preview refreshed.")

refresh_preview_btn.on_click(_refresh_preview)

def _prepare_artifacts_and_paths(qc: dict) -> tuple[Path, Path]:
    sel = get_selection()
    if sel is None:
        raise RuntimeError("No microscope selected.")
    year = qc.get("started_utc", AW.utc_now_iso())[:4]
    session_dir = repo_root / "qc" / "sessions" / sel.metadata.instrument_id / year
    artifacts_dir = session_dir / "artifacts" / qc["qc_id"]
    return session_dir, artifacts_dir

def _save(_):
    with save_out:
        clear_output()
        if state["qc"] is None:
            print("Initialize a QC session first.")
            return
        qc = state["qc"]

        # Ensure evaluation is blank (CI-only)
        qc["evaluation"] = qc.get("evaluation") or {}
        qc["evaluation"].setdefault("benchmark_ref", "")
        qc["evaluation"].setdefault("evaluated_utc", "")
        qc["evaluation"].setdefault("evaluated_by", "")
        qc["evaluation"].setdefault("overall_status", "")
        qc["evaluation"].setdefault("results", [])
        qc["evaluation"]["evaluated_utc"] = ""
        qc["evaluation"]["evaluated_by"] = ""
        qc["evaluation"]["overall_status"] = ""

        session_dir, artifacts_dir = _prepare_artifacts_and_paths(qc)
        session_dir.mkdir(parents=True, exist_ok=True)
        artifacts_dir.mkdir(parents=True, exist_ok=True)

        # 1) Save any uploaded CSVs from laser editors and register them as artifacts
        qc.setdefault("artifacts", [])
        uploaded = state.get("uploaded_files", {}) or {}
        for suggested_name, content in uploaded.items():
            safe_name = Path(suggested_name).name  # avoid accidental subpaths
            art_id = "art_" + slugify(Path(safe_name).stem)

            out_path = artifacts_dir / safe_name
            out_path.write_bytes(content)

            qc["artifacts"] = [a for a in qc["artifacts"] if a.get("artifact_id") != art_id]
            qc["artifacts"].append(
                {
                    "artifact_id": art_id,
                    "role": "table",
                    "uri": str(out_path.relative_to(repo_root)).replace(os.sep, "/"),
                    "description": f"Uploaded CSV ({safe_name})",
                }
            )

            # Rewrite any references inside laser_inputs_human (template expects artifact_id)
            lih = qc.get("laser_inputs_human", {})
            for block_name in ["linearity_series", "stability_series"]:
                for entry in (lih.get(block_name) or []):
                    if entry.get("csv_artifact") == suggested_name or entry.get("csv_artifact") == safe_name:
                        entry["csv_artifact"] = art_id

        # 2) Save image upload (optional)
        img = state.get("image_upload")
        if img:
            fn, content = img
            fn = Path(fn).name
            art_id = "art_target_image"
            out_path = artifacts_dir / fn
            out_path.write_bytes(content)

            # Update artifact uri
            for a in qc["artifacts"]:
                if a.get("artifact_id") == art_id:
                    a["uri"] = str(out_path.relative_to(repo_root)).replace(os.sep, "/")

        # 3) Save YAML
        yaml_path = session_dir / f"{qc['qc_id']}.yaml"
        save_yaml(yaml_path, qc)

        preview.value = _yaml_preview_text(qc)
        print(f"Saved QC YAML: {yaml_path.relative_to(repo_root)}")
        if uploaded:
            print(f"Saved {len(uploaded)} uploaded CSV artifact(s) under: {artifacts_dir.relative_to(repo_root)}")
        if img:
            print(f"Saved image artifact under: {artifacts_dir.relative_to(repo_root)}")

save_btn.on_click(_save)

git_panel = AW.git_commit_push_panel(
    repo_root,
    is_ready=lambda: state.get("qc") is not None,
    default_branch=lambda: (f"add-qc-{state['qc']['qc_id']}" if state.get("qc") else "add-qc"),
    default_commit_message=lambda: (f"Add QC session {state['qc']['qc_id']}" if state.get("qc") else "Add QC session"),
    header="Optional GitHub operations",
)

display(W.VBox([
    W.HBox([refresh_preview_btn, save_btn]),
    preview,
    save_out,
    git_panel,
]))
